In [56]:
# Call the relevant python packages
import numpy as np
import glob, os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.io import ascii

from sedfitter import filter_output, write_parameters, write_parameter_ranges
from sedfitter.extinction import Extinction
from sedfitter import fit, plot

import seaborn as sns
sns.set_context("paper")

%matplotlib inline

In [53]:
# Read in data table
file = '584_flux.csv'
data = ascii.read(file, data_start=1)

# Print the number of potential YSOs found in the table
YSO_num=len(data)
print("The number of YSO candidates in the table is",YSO_num)

for i in range(YSO_num):
    if data['J flux err'][i]!=1:
        data['J flux err'][i] = data['J flux err'][i]*2
    if data['H flux err'][i]!=1:
        data['H flux err'][i] = data['H flux err'][i]*2
    if data['K flux err'][i]!=1:
        data['K flux err'][i] = data['K flux err'][i]*2
    if data['36 flux err'][i]!=1:
        data['36 flux err'][i] = data['36 flux err'][i]*2
    if data['58 flux err'][i]!=1:
        data['58 flux err'][i] = data['58 flux err'][i]*3
    if data['80 flux err'][i]!=1:
        data['80 flux err'][i] = data['80 flux err'][i]*4
    if data['24 flux err'][i]!=1:
        data['24 flux err'][i] = data['24 flux err'][i]*4

#data.show_in_notebook()

The number of YSO candidates in the table is 584


In [54]:
# Write a column for an index and assign the flag
def write_column(idx,name,flag):
    for i in range(len(idx[0])):
        # If source is upper limit write 3 flag
        # If source is bad photometry write 0 flag
        data[name][idx[0][i]]=flag
        
# Set 8 and 24 as UL for SPs
data['80 flux err']=1.0
data['80 int']=3
data['24 flux err']=1.0
data['24 int']=3

#data.show_in_notebook()

# Find the total number of bands (max.8) with valid photometry
data['Total bands']=0
data['Total bands']=data['SpNum']+data['HJKNum']
# Filter sources leaving those with good photometry in at least 5 of 8 bands
idx_good_number_bands=np.where(data['Total bands']>=5)
num_good_number_bands=len(idx_good_number_bands[0])
print("Out of the above YSOs the number of sources with at least 5 of 8 bands is",num_good_number_bands)
# Create catalogue of these filtered sources to pass to next section
cat_good_number_bands=data[idx_good_number_bands]

# Write out table in the right format for SED fitter
ascii.write(cat_good_number_bands[['New name','RA','Dec','J int','H int','K int','36 int','45 int','58 int',
                                   '80 int','24 int','J flux','J flux err','H flux','H flux err','K flux',
                                   'K flux err','36 flux','36 flux err','45 flux','45 flux err','58 flux',
                                   '58 flux err','80 flux','80 flux err','24 flux','24 flux err']],
                                   '518_SP_824UL.txt', format='no_header', fast_writer=False,overwrite=True)

Out of the above YSOs the number of sources with at least 5 of 8 bands is 518


In [59]:
# Write a column for an index and assign the flag
def write_column(idx,name,flag):
    for i in range(len(idx)):
        # If source is upper limit write 3 flag
        # If source is bad photometry write 0 flag
        data[name][idx[i]]=flag

'''
good_YSO_path = '/Disk/ds-sopa-ifa-ug/s1625982/fred/ZIP/Good_YSOs/'
onlyfiles = [f for f in listdir(good_YSO_path) if isfile(join(good_YSO_path, f))]
good_YSO_array = np.array([int(onlyfiles[0][0:len(onlyfiles[0])-4])-1])
for i in range(1,len(onlyfiles)):
    filename = onlyfiles[i]
    num_file = int(filename[0:len(filename)-4])
    # take away 1 as 1.pdf corresponds to row 0
    new_array = np.array(num_file-1)
    good_YSO_array = np.append(good_YSO_array,new_array)
print("Number of files in the Good_YSO folder:",len(good_YSO_array))
data['Good_YSO'] = 0
write_column(good_YSO_array,'Good_YSO',1)


bad_YSO_path = '/Disk/ds-sopa-ifa-ug/s1625982/fred/ZIP/Bad_YSOs/'
onlyfiles = [f for f in listdir(bad_YSO_path) if isfile(join(bad_YSO_path, f))]
bad_YSO_array = np.array([int(onlyfiles[0][0:len(onlyfiles[0])-4])-1])
for i in range(1,len(onlyfiles)):
    filename = onlyfiles[i]
    num_file = int(filename[0:len(filename)-4])
    # take away 1 as 1.pdf corresponds to row 0
    new_array = np.array(num_file-1)
    bad_YSO_array = np.append(bad_YSO_array,new_array)
print("Number of files in the Bad_YSO folder:",len(bad_YSO_array))
data['Bad_YSO'] = 0
write_column(bad_YSO_array,'Bad_YSO',1)


poss_YSO_path = '/Disk/ds-sopa-ifa-ug/s1625982/fred/ZIP/Possible_YSOs/'
onlyfiles = [f for f in listdir(poss_YSO_path) if isfile(join(poss_YSO_path, f))]
poss_YSO_array = np.array([int(onlyfiles[0][0:len(onlyfiles[0])-4])-1])
for i in range(1,len(onlyfiles)):
    filename = onlyfiles[i]
    num_file = int(filename[0:len(filename)-4])
    # take away 1 as 1.pdf corresponds to row 0
    new_array = np.array(num_file-1)
    poss_YSO_array = np.append(poss_YSO_array,new_array)
print("Number of files in the Possible_YSO folder:",len(poss_YSO_array))
data['Possible_YSO'] = 0
write_column(poss_YSO_array,'Possible_YSO',1)
'''

good_SP_path = 'new_SP/good/'
onlyfiles = [f for f in listdir(good_SP_path) if isfile(join(good_SP_path, f))]
good_SP_array = np.array([int(onlyfiles[0][0:len(onlyfiles[0])-4])-1])
for i in range(1,len(onlyfiles)):
    filename = onlyfiles[i]
    num_file = int(filename[0:len(filename)-4])
    # take away 1 as 1.pdf corresponds to row 0
    new_array = np.array(num_file-1)
    good_SP_array = np.append(good_SP_array,new_array)
print("Number of files in the sp_good folder:",len(good_SP_array))
data['sp_good'] = 0
write_column(good_SP_array,'sp_good',1)


bad_SP_path = 'new_SP/bad'
onlyfiles = [f for f in listdir(bad_SP_path) if isfile(join(bad_SP_path, f))]
bad_SP_array = np.array([int(onlyfiles[0][0:len(onlyfiles[0])-4])-1])
for i in range(1,len(onlyfiles)):
    filename = onlyfiles[i]
    num_file = int(filename[0:len(filename)-4])
    # take away 1 as 1.pdf corresponds to row 0
    new_array = np.array(num_file-1)
    bad_SP_array = np.append(bad_SP_array,new_array)
print("Number of files in the sp_bad folder:",len(bad_SP_array))
data['sp_bad'] = 0
write_column(bad_SP_array,'sp_bad',1)


poss_SP_path = 'new_SP/poss/'
onlyfiles = [f for f in listdir(poss_SP_path) if isfile(join(poss_SP_path, f))]
poss_SP_array = np.array([int(onlyfiles[0][0:len(onlyfiles[0])-4])-1])
for i in range(1,len(onlyfiles)):
    filename = onlyfiles[i]
    num_file = int(filename[0:len(filename)-4])
    # take away 1 as 1.pdf corresponds to row 0
    new_array = np.array(num_file-1)
    poss_SP_array = np.append(poss_SP_array,new_array)
print("Number of files in the Possible_SP folder:",len(poss_SP_array))
data['Possible_SP'] = 0
write_column(poss_SP_array,'Possible_SP',1)

data.show_in_notebook()

Number of files in the sp_good folder: 256
Number of files in the sp_bad folder: 219
Number of files in the Possible_SP folder: 37


idx,ID,RA,Dec,k36mag,e36mag,d36mag,k45mag,e45mag,d45mag,k58mag,e58mag,d58mag,k80mag,e80mag,d80mag,k24mag,e24mag,dstArcSec,RAJ2000,DEJ2000,Jmag,eJmag,Hmag,eHmag,Kmag,eKmag,c1a,c1b,c1c,c2a,c2b,c2c,c3a,c3b,c3c,c4a,c4b,c4c,c5a,c5b,c5c,c6a,c6b,c6c,c7,c8,c9a,c9b,c9c,c11b,c11c,c1,c2,c3,c4,c5,c6,c9,c10,c11,24 data?,36 data?,45 data?,58 data?,80 data?,SpNum,H data?,J data?,K data?,HJKNum,c2 met?,c3 met?,c4 met?,c5 met?,c6 met?,c9 met?,c11 met?,sp rel?,HJK rel?,YSO Score,24 without 58/80,New name,J flux,H flux,K flux,36 flux,45 flux,58 flux,80 flux,24 flux,J flux err,H flux err,K flux err,36 flux err,45 flux err,58 flux err,80 flux err,24 flux err,J int,H int,K int,36 int,45 int,58 int,80 int,24 int,Total bands,sp_good,sp_bad,Possible_SP
0,3776,296.02927,-14.68646,17.16,0.05,0.12,16.14,0.05,-0.04,15.31,0.08,0.48,14.02,0.07,0.06,10.02,0.04,0.715527028,296.029144,-14.686303,19.31,0.164,18.867,0.131,18.045,0.089,1,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,3,1,1,1,1,1,1,0,0,1,1,1,1,1,5,1,1,1,3,1,1,1,1,1,1,0,1,1,6,0,1,0.02978457,0.029848364,0.040876078,0.038419503,0.062885147,0.086436633,2.549884675,6.37154415,0.005956914,0.005969672,0.008175216,0.0076839,0.006288515,0.025930988999999998,1.0,1.0,1,1,1,1,1,1,3,3,8,0,1,0
1,8145,296.04298,-14.80325,17.14,0.11,1.04,17.04,0.16,1.26,15.96,0.15,0.83,13.55,0.08,1.15,9.05,0.03,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,5,0,0,0,0,0,0,1,1,1,1,0,1,0,4,0,2,inf,inf,inf,0.039133775,0.027450362,0.047500463,3.931158358,15.56842564,inf,inf,inf,0.007826754,0.002745036,0.014250137999999999,1.0,1.0,0,0,0,1,1,1,3,3,5,0,0,1
2,2235,296.04335,-14.80379,15.89,0.08,0.83,15.55,0.09,0.83,15.38,0.12,1.23,12.4,0.04,0.61,8.95,0.03,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,5,0,0,0,0,0,0,1,1,1,1,0,1,0,4,0,3,inf,inf,inf,0.123751861,0.108279958,0.081039703,11.33758455,17.07043927,inf,inf,inf,0.024750372,0.010827996,0.02431191,1.0,1.0,0,0,0,1,1,1,3,3,5,1,0,0
3,3160,296.04509,-14.67401,16.32,0.14,-9999.0,15.94,0.12,1.19,10.41,0.12,-3.87,10.43,0.05,-3.12,12.28,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,1,0,1,0,1,1,1,1,4,0,0,0,0,1,1,1,0,0,1,1,1,0,4,0,4,inf,inf,inf,0.083282114,0.075604575,7.883114604,69.58578626,0.794775913,inf,inf,inf,0.016656422,0.007560458,2.36493438,1.0,1.0,0,0,0,1,1,1,3,3,4,0,0,0
4,19111,296.04821,-14.68252,18.83,0.1,-0.32,18.01,0.09,0.1,16.28,0.18,-9999.0,14.97,0.08,-0.93,11.27,0.08,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,1,1,1,1,5,0,0,0,0,1,0,1,0,0,1,0,1,0,3,0,5,inf,inf,inf,0.008251858,0.011234353,0.035375113,1.062968852,2.014859173,inf,inf,inf,0.001650372,0.001123435,0.010612533,1.0,1.0,0,0,0,1,1,1,3,3,5,0,1,0
5,6762,296.04945,-14.84182,17.82,0.17,1.13,16.8,0.1,0.56,15.95,0.12,-0.59,14.8,0.08,0.01,11.36,0.21,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,1,1,1,4,0,0,0,0,1,0,1,0,0,1,0,1,0,3,0,6,inf,inf,inf,0.020919521,0.034241129,0.047939979,1.243141425,1.854576263,inf,inf,inf,0.004183904,0.003424113,0.014381993999999999,1.0,1.0,0,0,0,1,1,1,3,3,4,0,0,0
6,3480,296.05655,-14.87977,17.01,0.15,0.79,16.05,0.09,0.06,15.69,0.12,-0.1,15.36,0.13,0.1,9.88,0.04,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,1,1,1,5,0,0,0,0,1,0,0,1,1,0,0,1,0,3,0,7,inf,inf,inf,0.044111491,0.068320035,0.060911296,0.742199297,7.248442478,inf,inf,inf,0.008822298,0.006832003,0.01827339,1.0,1.0,0,0,0,1,1,1,3,3,5,1,0,0
7,2491,296.05716,-14.83338,16.11,0.11,0.25,15.68,0.12,0.3,15.89,0.09,0.66,13.55,0.05,0.21,9.92,0.04,-9999.0,-9999.0,-9999.

In [50]:
# Define path to models
model_dir = 'models_SP/'

# Read in extinction law
extinction = Extinction.from_file('Input_SED/kmh94.par',columns=[0, 3], wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

# Define filters and apertures
filters = ['2J', '2H', '2K', 'I1', 'I2', 'I3', 'I4', 'M1']
apertures = [2., 2., 2., 1.2, 1.2, 1.2, 1.2, 6.] * u.arcsec


# Run the fitting
# Creates an output file which can be saved and plotted later
# YSO 2006 models take a long time to run - too long for the kernel to run in one go.
# Define start and end sources to run - the user must have trimmed the master table to the appropriate limits
#SED_input_file = '/scratch/s1625982/NGC6822/env/Input_SED/YSO_No_Flag'+start_source+'to'+end_source+'.txt'
SED_input_file = 'extra2.prn'
#SED_output_file = '/scratch/s1625982/NGC6822/env/Output_SED/YSO_2006/YSO_No_Flag'+start_source+'to'+end_source+'.fitinfo'
SED_output_file = 'extra2.fitinfo'
data = ascii.read(SED_input_file)
fit(SED_input_file, filters, apertures, model_dir,
    SED_output_file,
    extinction_law=extinction,
    distance_range=[450., 530.] * u.kpc,     # Distance range here is for NGC 6822
    av_range=[0., 40.])                      # A_V range here is for NGC 6822

 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  Kurucz stellar photospheres
   Log[d] stepping     :  0.02

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading models_SP//convolved/2J.fits
   Reading models_SP//convolved/2H.fits
   Reading models_SP//convolved/2K.fits
   Reading models_SP//convolved/I1.fits
   Reading models_SP//convolved/I2.fits
   Reading models_SP//convolved/I3.fits
   Reading models_SP//convolved/I4.fits
   Reading models_SP//convolved/M1.fits
 ------------------------------------------------------------
  => Fitting parameters
 ------------------------------------------------------------

   Minimum A_V      :     0.000 mag
   Maximum A_V      :    40.000 mag
   Minimum distance :   450.000 kpc
   Maximum distance :   530.000 

In [51]:
# Plot the fits for the output file
plot(SED_output_file, 'SP_new')

Delete directory SP_new? [y/[n]] y

